In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import json
import random
import tensorflow as tf
import pickle

with open ("intents.json") as file:
    data = json.load(file)
    
#print (data["intents"])    
#with open ('data.pickle',"rb") as D:
#    db = pickle.load(D)
#    print(D)
try:
    x
    with open("data.pickle","rb") as f:
        words, labels, train, output = pickle.load(f)
    
except:
    words = []
    labels = []
    doc_x = []
    doc_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            doc_x.append(wrds)
            doc_y.append(intent["tag"])
        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    train = []
    output = []

    out_empty = [ 0 for _ in range(len(labels))]

    for x, doc in enumerate(doc_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row =  out_empty[:]
        output_row[labels.index(doc_y[x])] = 1

        train.append(bag)
        output.append(output_row)


    train = numpy.array(train)
    output = numpy.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, train, output),f)


#tf.reset_default_graph()

net = tflearn.input_data(shape = [None, len(train[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)
model = tflearn.DNN(net)

#try:
#    model.load("model.tflearn")
    
#except:
model.fit(train, output, n_epoch = 3000, batch_size = 8, show_metric = True)
model.save("model.tflearn")

    
    
def bag_of_words(s,words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return numpy.array(bag)


def chat():
    print("@@@@@@ ASK APRIL SOMETHING! @@@@@@")
    print("~~~~~~~ To exit type quit ~~~~~~~")
    
    while True:
        ip = input("YOU:")
        if ip.lower() == "quit":
            break
        
        results =  model.predict([bag_of_words(ip, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(random.choice(responses))
        
        
        
chat()



Training Step: 11999  | total loss: 0.74682 | time: 0.012s
| Adam | epoch: 3000 | loss: 0.74682 - acc: 0.8512 -- iter: 24/31
Training Step: 12000  | total loss: 0.68322 | time: 0.017s
| Adam | epoch: 3000 | loss: 0.68322 - acc: 0.8661 -- iter: 31/31
--
INFO:tensorflow:C:\Users\vk\Desktop\Git Projects\AI_Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
@@@@@@ ASK APRIL SOMETHING! @@@@@@
~~~~~~~ To exit type quit ~~~~~~~
YOU:hey
Hello!
YOU:Hi
Good to see you again!
YOU:bb
Goodbye!
YOU:Bye
Talk to you later
YOU:What is you name?
I'm April!
YOU:name?
My name is April!
YOU:What can you do for me?
I can talk to you
YOU:What is your function?
Hi there, how can I help?
YOU:What is your job?
Hi there, how can I help?
YOU:whats your purpose?
I can just talk to you and give you some information
YOU:what do can you do?
I can tell you a joke or ask you a riddle
YOU:crack a joke?
What do you call a boomerang that doesn't work? A stick.
YOU:make me laugh
What did Neil A